In [1]:
import sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from copy import deepcopy
from torch.nn.parallel import DistributedDataParallel as DDP, DataParallel as DP

%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
np.__version__

'1.20.3'

In [3]:
# import das.pytorch.estimator as pe
import torch
torch.__version__

'1.8.1+cu111'

In [4]:
torch.cuda.is_available()

True

In [5]:
# sys.path.insert(0,"/media/cfs/wenhao71/workspace/torch_ecg/")
sys.path.insert(0, "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/")
from train.train_seq_lab_net_cpsc2019.cfg import TrainCfg, ModelCfg
from train.train_seq_lab_net_cpsc2019.train import train, evaluate
from train.train_seq_lab_net_cpsc2019.model import ECG_SEQ_LAB_NET_CPSC2019
from train.train_seq_lab_net_cpsc2019.utils import init_logger, dict_to_str


In [6]:
TrainCfg.db_dir = "/home/wenh06/Jupyter/data/CPSC2019/"
TrainCfg

{'fs': 500,
 'db_dir': '/home/wenh06/Jupyter/data/CPSC2019/',
 'log_dir': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/log',
 'checkpoints': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/checkpoints',
 'model_dir': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/saved_models',
 'final_model_name': None,
 'keep_checkpoint_max': 20,
 'train_ratio': 0.8,
 'input_len': 5000,
 'classes': ['N'],
 'n_leads': 1,
 'bias_thr': 37.5,
 'skip_dist': 250.0,
 'label_smoothing': 0.0,
 'random_normalize': True,
 'random_normalize_mean': [-0.05, 0.1],
 'random_normalize_std': [0.08, 0.32],
 'baseline_wander': True,
 'bw': True,
 'bw_fs': array([0.33, 0.1 , 0.05, 0.01]),
 'bw_ampl_ratio': array([[0.01, 0.01, 0.02, 0.03],
        [0.01, 0.02, 0.04, 0.05],
        [0.1 , 0.06, 0.04, 0.02],
        [0.02, 0.04, 0.07, 0.1 ],
        [0.05, 0.1 , 0.16, 0.25],
        [0.1 , 0.15, 0.25, 0.3 ],
        [0.

In [7]:
ModelCfg

{'skip_dist': 250.0,
 'torch_dtype': 'float',
 'fs': 500,
 'spacing': 2.0,
 'classes': ['N'],
 'n_leads': 1,
 'seq_lab_crnn': {'fs': 500,
  'classes': ['N'],
  'n_leads': 1,
  'skip_dist': 250.0,
  'torch_dtype': 'float',
  'cnn': {'name': 'multi_scopic',
   'multi_scopic': {'groups': 1,
    'scopes': [[[1], [1, 1], [1, 1, 1]],
     [[2], [2, 4], [8, 8, 8]],
     [[4], [4, 8], [16, 32, 64]]],
    'filter_lengths': [[11, 7, 5], [11, 7, 5], [11, 7, 5]],
    'subsample_lengths': [2, 2, 2],
    'num_filters': [[32, 64, 128], [32, 64, 128], [32, 64, 128]],
    'dropouts': [[0, 0.2, 0], [0, 0.2, 0], [0, 0.2, 0]],
    'bias': True,
    'kernel_initializer': 'he_normal',
    'kw_initializer': {},
    'activation': 'relu',
    'kw_activation': {'inplace': True},
    'block': {'subsample_mode': 'max',
     'bias': True,
     'kernel_initializer': 'he_normal',
     'kw_initializer': {},
     'activation': 'relu',
     'kw_activation': {'inplace': True},
     'batch_norm': True,
     'kw_bn': {}}}

# train CNN

In [8]:
train_config = deepcopy(TrainCfg)

In [9]:
train_config.model_name = "cnn"

In [10]:
model_config = deepcopy(ModelCfg[f"seq_lab_{train_config.model_name.lower()}"])

In [11]:
train_config

{'fs': 500,
 'db_dir': '/home/wenh06/Jupyter/data/CPSC2019/',
 'log_dir': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/log',
 'checkpoints': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/checkpoints',
 'model_dir': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/saved_models',
 'final_model_name': None,
 'keep_checkpoint_max': 20,
 'train_ratio': 0.8,
 'input_len': 5000,
 'classes': ['N'],
 'n_leads': 1,
 'bias_thr': 37.5,
 'skip_dist': 250.0,
 'label_smoothing': 0.0,
 'random_normalize': True,
 'random_normalize_mean': [-0.05, 0.1],
 'random_normalize_std': [0.08, 0.32],
 'baseline_wander': True,
 'bw': True,
 'bw_fs': array([0.33, 0.1 , 0.05, 0.01]),
 'bw_ampl_ratio': array([[0.01, 0.01, 0.02, 0.03],
        [0.01, 0.02, 0.04, 0.05],
        [0.1 , 0.06, 0.04, 0.02],
        [0.02, 0.04, 0.07, 0.1 ],
        [0.05, 0.1 , 0.16, 0.25],
        [0.1 , 0.15, 0.25, 0.3 ],
        [0.

In [12]:
model_config

{'fs': 500,
 'classes': ['N'],
 'n_leads': 1,
 'skip_dist': 250.0,
 'torch_dtype': 'float',
 'cnn': {'name': 'multi_scopic',
  'multi_scopic': {'groups': 1,
   'scopes': [[[1], [1, 1], [1, 1, 1]],
    [[2], [2, 4], [8, 8, 8]],
    [[4], [4, 8], [16, 32, 64]]],
   'filter_lengths': [[11, 7, 5], [11, 7, 5], [11, 7, 5]],
   'subsample_lengths': [2, 2, 2],
   'num_filters': [[32, 64, 128], [32, 64, 128], [32, 64, 128]],
   'dropouts': [[0, 0.2, 0], [0, 0.2, 0], [0, 0.2, 0]],
   'bias': True,
   'kernel_initializer': 'he_normal',
   'kw_initializer': {},
   'activation': 'relu',
   'kw_activation': {'inplace': True},
   'block': {'subsample_mode': 'max',
    'bias': True,
    'kernel_initializer': 'he_normal',
    'kw_initializer': {},
    'activation': 'relu',
    'kw_activation': {'inplace': True},
    'batch_norm': True,
    'kw_bn': {}}}},
 'rnn': {'name': 'none'},
 'attn': {'name': 'se',
  'se': {'reduction': 8,
   'activation': 'relu',
   'kw_activation': {'inplace': True},
   'bias':

In [13]:
model = ECG_SEQ_LAB_NET_CPSC2019(
    n_leads=train_config.n_leads,
    input_len=train_config.input_len,
    config=model_config,
)

model_config = {'fs': 500, 'classes': ['N'], 'n_leads': 1, 'skip_dist': 250.0, 'torch_dtype': 'float', 'cnn': {'name': 'multi_scopic', 'multi_scopic': {'groups': 1, 'scopes': [[[1], [1, 1], [1, 1, 1]], [[2], [2, 4], [8, 8, 8]], [[4], [4, 8], [16, 32, 64]]], 'filter_lengths': [[11, 7, 5], [11, 7, 5], [11, 7, 5]], 'subsample_lengths': [2, 2, 2], 'num_filters': [[32, 64, 128], [32, 64, 128], [32, 64, 128]], 'dropouts': [[0, 0.2, 0], [0, 0.2, 0], [0, 0.2, 0]], 'bias': True, 'kernel_initializer': 'he_normal', 'kw_initializer': {}, 'activation': 'relu', 'kw_activation': {'inplace': True}, 'block': {'subsample_mode': 'max', 'bias': True, 'kernel_initializer': 'he_normal', 'kw_initializer': {}, 'activation': 'relu', 'kw_activation': {'inplace': True}, 'batch_norm': True, 'kw_bn': {}}}}, 'rnn': {'name': 'none'}, 'attn': {'name': 'se', 'se': {'reduction': 8, 'activation': 'relu', 'kw_activation': {'inplace': True}, 'bias': True, 'kernel_initializer': 'he_normal'}}, 'clf': {'out_channels': [256, 

In [14]:
torch.cuda.device_count()

2

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [16]:
if torch.cuda.device_count() > 1:
    model = DP(model)
#     model = DDP(model)

model.to(device=device)
model.__DEBUG__ = False

In [17]:
logger = init_logger(log_dir=train_config.log_dir, verbose=2)
logger.info(f"\n{'*'*20}   Start Training   {'*'*20}\n")
logger.info(f'Using device {device}')
logger.info(f"Using torch of version {torch.__version__}")
logger.info(f'with configuration\n{dict_to_str(train_config)}')

log file path: /home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/log/log_2021-06-22-09-31-50.txt
levels of c_handler and f_handler are set DEBUG
ECG-SEQ-LAB - INFO - 
********************   Start Training   ********************

ECG-SEQ-LAB - INFO - Using device cuda
ECG-SEQ-LAB - INFO - Using torch of version 1.8.1+cu111
ECG-SEQ-LAB - INFO - with configuration
{
    "fs": 500
    "db_dir": "/home/wenh06/Jupyter/data/CPSC2019/"
    "log_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/log"
    "checkpoints": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/checkpoints"
    "model_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/saved_models"
    "final_model_name": None
    "keep_checkpoint_max": 20
    "train_ratio": 0.8
    "input_len": 5000
    "classes": [
        "N"
    ]
    "n_leads": 1
    "bias_thr": 37.5
    "skip_dist": 250.0
    "lab

In [18]:
# hehe_input = torch.rand((2,1,4000)).to(device,dtype=torch.float32)

In [19]:
# pred, rpeaks = model.inference(hehe_input,0.5)

In [20]:
# rpeaks

In [21]:
train_config.debug = True
train_config.cnn_name = model_config.cnn.name
train_config.rnn_name = model_config.rnn.name
train_config.attn_name = model_config.attn.name
train_config

{'fs': 500,
 'db_dir': '/home/wenh06/Jupyter/data/CPSC2019/',
 'log_dir': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/log',
 'checkpoints': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/checkpoints',
 'model_dir': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/saved_models',
 'final_model_name': None,
 'keep_checkpoint_max': 20,
 'train_ratio': 0.8,
 'input_len': 5000,
 'classes': ['N'],
 'n_leads': 1,
 'bias_thr': 37.5,
 'skip_dist': 250.0,
 'label_smoothing': 0.0,
 'random_normalize': True,
 'random_normalize_mean': [-0.05, 0.1],
 'random_normalize_std': [0.08, 0.32],
 'baseline_wander': True,
 'bw': True,
 'bw_fs': array([0.33, 0.1 , 0.05, 0.01]),
 'bw_ampl_ratio': array([[0.01, 0.01, 0.02, 0.03],
        [0.01, 0.02, 0.04, 0.05],
        [0.1 , 0.06, 0.04, 0.02],
        [0.02, 0.04, 0.07, 0.1 ],
        [0.05, 0.1 , 0.16, 0.25],
        [0.1 , 0.15, 0.25, 0.3 ],
        [0.

In [22]:
# train(
#     model=model,
#     config=train_config,
#     device=device,
#     logger=logger,
#     debug=train_config.debug,
# )
best_state_dict = train(
    model=model,
    model_config=model_config,
    device=device,
    config=train_config,
    logger=logger,
    debug=train_config.debug,
)

ECG-SEQ-LAB - INFO - training configurations are as follows:
{
    "fs": 500
    "db_dir": "/home/wenh06/Jupyter/data/CPSC2019/"
    "log_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/log"
    "checkpoints": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/checkpoints"
    "model_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/saved_models"
    "final_model_name": None
    "keep_checkpoint_max": 20
    "train_ratio": 0.8
    "input_len": 5000
    "classes": [
        "N"
    ]
    "n_leads": 1
    "bias_thr": 37.5
    "skip_dist": 250.0
    "label_smoothing": 0.0
    "random_normalize": True
    "random_normalize_mean": [
        -0.05, 0.1
    ]
    "random_normalize_std": [
        0.08, 0.32
    ]
    "baseline_wander": True
    "bw": True
    "bw_fs": [0.33 0.1  0.05 0.01]
    "bw_ampl_ratio": [[0.01 0.01 0.02 0.03]
 [0.01 0.02 0.04 0.05]
 [0.1  0.06 0.04 0.02]
 [0.02 

Epoch 1/300:  96%|███████████████████████████████████████████▏ | 1536/1600 [00:05<00:00, 666.72it/s]

QRS_acc: 0.4031
Scoring complete.
QRS_acc: 0.3918
Scoring complete.
ECG-SEQ-LAB - INFO - 
Train epoch_1:
--------------------
train/epoch_loss:        2.724177971482277
train/qrs_score:         0.4031

test/qrs_score:          0.3918
---------------------------------

ECG-SEQ-LAB - INFO - Checkpoint 1 saved!


Epoch 2/300:  96%|██████████████████████████████████████████▏ | 1536/1600 [00:01<00:00, 1254.56it/s]

QRS_acc: 0.1969
Scoring complete.
QRS_acc: 0.1775
Scoring complete.
ECG-SEQ-LAB - INFO - 
Train epoch_2:
--------------------
train/epoch_loss:        1.387441709637642
train/qrs_score:         0.1969

test/qrs_score:          0.1775
---------------------------------

ECG-SEQ-LAB - INFO - Checkpoint 2 saved!


...

ECG-SEQ-LAB - INFO - Train step_920: loss : 0.03156221657991409


Epoch 132/300:  96%|████████████████████▏| 1536/1600 [00:01<00:00, 1245.96it/s, loss (batch)=0.0316]

QRS_acc: 0.9784
Scoring complete.
QRS_acc: 0.9028
Scoring complete.
ECG-SEQ-LAB - INFO - 
Train epoch_132:
--------------------
train/epoch_loss:        0.2097537610679865
train/qrs_score:         0.9784

test/qrs_score:          0.9028
---------------------------------

ECG-SEQ-LAB - INFO - early stopping is triggered at epoch 132


Epoch 132/300: 100%|██████████████████████| 1600/1600 [00:05<00:00, 318.53it/s, loss (batch)=0.0316]

ECG-SEQ-LAB - INFO - Best model saved to /home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/saved_models/ECG_SEQ_LAB_NET_CPSC2019_multi_scopic_none_epoch_2021-06-22-09-43-13_BestModel_challenge_score(qrs_score)_0.9095.pth.tar!


In [23]:
# best_state_dict

# train CRNN

In [24]:
train_config = deepcopy(TrainCfg)
train_config.model_name = "crnn"
model_config = deepcopy(ModelCfg[f"seq_lab_{train_config.model_name.lower()}"])

In [25]:
model = ECG_SEQ_LAB_NET_CPSC2019(
    n_leads=train_config.n_leads,
    input_len=train_config.input_len,
    config=model_config,
)

if torch.cuda.device_count() > 1:
    model = DP(model)
#     model = DDP(model)

model.to(device=device)
model.__DEBUG__ = False

model_config = {'fs': 500, 'classes': ['N'], 'n_leads': 1, 'skip_dist': 250.0, 'torch_dtype': 'float', 'cnn': {'name': 'multi_scopic', 'multi_scopic': {'groups': 1, 'scopes': [[[1], [1, 1], [1, 1, 1]], [[2], [2, 4], [8, 8, 8]], [[4], [4, 8], [16, 32, 64]]], 'filter_lengths': [[11, 7, 5], [11, 7, 5], [11, 7, 5]], 'subsample_lengths': [2, 2, 2], 'num_filters': [[32, 64, 128], [32, 64, 128], [32, 64, 128]], 'dropouts': [[0, 0.2, 0], [0, 0.2, 0], [0, 0.2, 0]], 'bias': True, 'kernel_initializer': 'he_normal', 'kw_initializer': {}, 'activation': 'relu', 'kw_activation': {'inplace': True}, 'block': {'subsample_mode': 'max', 'bias': True, 'kernel_initializer': 'he_normal', 'kw_initializer': {}, 'activation': 'relu', 'kw_activation': {'inplace': True}, 'batch_norm': True, 'kw_bn': {}}}}, 'rnn': {'name': 'lstm', 'lstm': {'hidden_sizes': [256, 64], 'bias': True, 'dropouts': 0.2, 'bidirectional': True}}, 'attn': {'name': 'se', 'se': {'reduction': 8, 'activation': 'relu', 'kw_activation': {'inplace

In [27]:
logger = init_logger(log_dir=train_config.log_dir, verbose=2)
logger.info(f"\n{'*'*20}   Start Training   {'*'*20}\n")
logger.info(f'Using device {device}')
logger.info(f"Using torch of version {torch.__version__}")
logger.info(f'with configuration\n{dict_to_str(train_config)}')

log file path: /home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/log/log_2021-06-22-09-43-13.txt
levels of c_handler and f_handler are set DEBUG
ECG-SEQ-LAB - INFO - 
********************   Start Training   ********************

ECG-SEQ-LAB - INFO - 
********************   Start Training   ********************

ECG-SEQ-LAB - INFO - Using device cuda
ECG-SEQ-LAB - INFO - Using device cuda
ECG-SEQ-LAB - INFO - Using torch of version 1.8.1+cu111
ECG-SEQ-LAB - INFO - Using torch of version 1.8.1+cu111
ECG-SEQ-LAB - INFO - with configuration
{
    "fs": 500
    "db_dir": "/home/wenh06/Jupyter/data/CPSC2019/"
    "log_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/log"
    "checkpoints": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/checkpoints"
    "model_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/saved_models"
    "final_model_name": None


In [28]:
train_config.debug = True
train_config.cnn_name = model_config.cnn.name
train_config.rnn_name = model_config.rnn.name
train_config.attn_name = model_config.attn.name
train_config

{'fs': 500,
 'db_dir': '/home/wenh06/Jupyter/data/CPSC2019/',
 'log_dir': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/log',
 'checkpoints': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/checkpoints',
 'model_dir': '/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/saved_models',
 'final_model_name': None,
 'keep_checkpoint_max': 20,
 'train_ratio': 0.8,
 'input_len': 5000,
 'classes': ['N'],
 'n_leads': 1,
 'bias_thr': 37.5,
 'skip_dist': 250.0,
 'label_smoothing': 0.0,
 'random_normalize': True,
 'random_normalize_mean': [-0.05, 0.1],
 'random_normalize_std': [0.08, 0.32],
 'baseline_wander': True,
 'bw': True,
 'bw_fs': array([0.33, 0.1 , 0.05, 0.01]),
 'bw_ampl_ratio': array([[0.01, 0.01, 0.02, 0.03],
        [0.01, 0.02, 0.04, 0.05],
        [0.1 , 0.06, 0.04, 0.02],
        [0.02, 0.04, 0.07, 0.1 ],
        [0.05, 0.1 , 0.16, 0.25],
        [0.1 , 0.15, 0.25, 0.3 ],
        [0.

In [29]:
best_state_dict = train(
    model=model,
    model_config=model_config,
    device=device,
    config=train_config,
    logger=logger,
    debug=train_config.debug,
)

ECG-SEQ-LAB - INFO - training configurations are as follows:
{
    "fs": 500
    "db_dir": "/home/wenh06/Jupyter/data/CPSC2019/"
    "log_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/log"
    "checkpoints": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/checkpoints"
    "model_dir": "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/saved_models"
    "final_model_name": None
    "keep_checkpoint_max": 20
    "train_ratio": 0.8
    "input_len": 5000
    "classes": [
        "N"
    ]
    "n_leads": 1
    "bias_thr": 37.5
    "skip_dist": 250.0
    "label_smoothing": 0.0
    "random_normalize": True
    "random_normalize_mean": [
        -0.05, 0.1
    ]
    "random_normalize_std": [
        0.08, 0.32
    ]
    "baseline_wander": True
    "bw": True
    "bw_fs": [0.33 0.1  0.05 0.01]
    "bw_ampl_ratio": [[0.01 0.01 0.02 0.03]
 [0.01 0.02 0.04 0.05]
 [0.1  0.06 0.04 0.02]
 [0.02 

Epoch 1/300:   0%|                                                         | 0/1600 [00:00<?, ?it/s]/home/wenh06/.local/lib/python3.8/site-packages/torch/nn/modules/rnn.py:661: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:915.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,
Epoch 1/300:  96%|███████████████████████████████████████████▏ | 1536/1600 [00:02<00:00, 815.87it/s]

QRS_acc: 0.3582
Scoring complete.
QRS_acc: 0.3703
Scoring complete.
ECG-SEQ-LAB - INFO - 
Train epoch_1:
--------------------
train/epoch_loss:        2.7665919363498688
train/qrs_score:         0.3582

test/qrs_score:          0.3703
---------------------------------

ECG-SEQ-LAB - INFO - 
Train epoch_1:
--------------------
train/epoch_loss:        2.7665919363498688
train/qrs_score:         0.3582

test/qrs_score:          0.3703
---------------------------------

ECG-SEQ-LAB - INFO - Checkpoint 1 saved!
ECG-SEQ-LAB - INFO - Checkpoint 1 saved!


Epoch 2/300:  96%|███████████████████████████████████████████▏ | 1536/1600 [00:02<00:00, 816.99it/s]

QRS_acc: 0.5559
Scoring complete.
QRS_acc: 0.532
Scoring complete.
ECG-SEQ-LAB - INFO - 
Train epoch_2:
--------------------
train/epoch_loss:        1.3083886355161667
train/qrs_score:         0.5559

test/qrs_score:          0.532
---------------------------------

ECG-SEQ-LAB - INFO - 
Train epoch_2:
--------------------
train/epoch_loss:        1.3083886355161667
train/qrs_score:         0.5559

test/qrs_score:          0.532
---------------------------------

ECG-SEQ-LAB - INFO - Checkpoint 2 saved!
ECG-SEQ-LAB - INFO - Checkpoint 2 saved!


...

ECG-SEQ-LAB - INFO - Train step_780: loss : 0.019217701628804207
ECG-SEQ-LAB - INFO - Train step_780: loss : 0.019217701628804207


Epoch 112/300:  96%|█████████████████████ | 1536/1600 [00:02<00:00, 810.79it/s, loss (batch)=0.0192]

QRS_acc: 0.9899
Scoring complete.
QRS_acc: 0.8988
Scoring complete.
ECG-SEQ-LAB - INFO - 
Train epoch_112:
--------------------
train/epoch_loss:        0.14465167745947838
train/qrs_score:         0.9899

test/qrs_score:          0.8988
---------------------------------

ECG-SEQ-LAB - INFO - 
Train epoch_112:
--------------------
train/epoch_loss:        0.14465167745947838
train/qrs_score:         0.9899

test/qrs_score:          0.8988
---------------------------------

ECG-SEQ-LAB - INFO - early stopping is triggered at epoch 112
ECG-SEQ-LAB - INFO - early stopping is triggered at epoch 112


Epoch 112/300: 100%|██████████████████████| 1600/1600 [00:06<00:00, 261.57it/s, loss (batch)=0.0192]

ECG-SEQ-LAB - INFO - Best model saved to /home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/saved_models/ECG_SEQ_LAB_NET_CPSC2019_multi_scopic_lstm_epoch_2021-06-22-09-54-54_BestModel_challenge_score(qrs_score)_0.9038.pth.tar!
ECG-SEQ-LAB - INFO - Best model saved to /home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/saved_models/ECG_SEQ_LAB_NET_CPSC2019_multi_scopic_lstm_epoch_2021-06-22-09-54-54_BestModel_challenge_score(qrs_score)_0.9038.pth.tar!


# DEBUG

In [31]:
model.module.load_state_dict(best_state_dict)

<All keys matched successfully>

In [32]:
from train.database_reader.database_reader.cpsc_databases.cpsc2019 import CPSC2019 as DR
from dataset import CPSC2019 as DG

In [33]:
dg = DG(TrainCfg)

log file path is set "/home/wenh06/Jupyter/wenhao/workspace/torch_ecg/train/train_seq_lab_net_cpsc2019/CPSC2019-CPSC2019.log"
levels of c_handler and f_handler are set DEBUG


In [34]:
values, labels = dg[2]

In [35]:
values.shape, labels.shape

((1, 5000), (625, 1))

In [36]:
values = values.astype(np.float32)

In [38]:
pred, rpeaks = model.module.inference(values[np.newaxis,...])

In [39]:
pred

array([[0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ,
        0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ,
        0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ,
        0.     , 0.     , 0.     , 0.00001, 0.00071, 0.17913, 0.94618,
        0.99898, 0.99997, 0.99992, 0.99987, 0.99997, 1.     , 1.     ,
        0.99997, 0.99633, 0.13221, 0.01585, 0.00002, 0.     , 0.     ,
        0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ,
        0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ,
        0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ,
        0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ,
        0.     , 0.     , 0.     , 0.     , 0.     , 0.     , 0.     ,
        0.     , 0.     , 0.     , 0.     , 0.00107, 0.33828, 0.90369,
        0.99997, 0.99999, 1.     , 1.     , 1.     , 1.     , 1.     ,
        1.     , 0.99993, 0.35735, 0.04375, 0.00002, 0.     , 0.     ,
      

In [40]:
pred.shape

(1, 625)

In [41]:
rpeaks

[array([ 256,  704, 1152, 1604, 2052, 2500, 2952, 3396, 3836, 4288, 4596])]

In [42]:
hehe_fwd = model.module.forward(torch.from_numpy(values[np.newaxis,...]).to(device))

In [43]:
hehe_fwd

tensor([[[-1.8846e+01],
         [-2.3426e+01],
         [-2.7113e+01],
         [-2.5824e+01],
         [-2.2551e+01],
         [-2.9579e+01],
         [-3.0723e+01],
         [-3.2809e+01],
         [-2.7633e+01],
         [-2.3200e+01],
         [-3.0879e+01],
         [-2.9089e+01],
         [-3.1438e+01],
         [-3.0890e+01],
         [-2.0101e+01],
         [-2.8980e+01],
         [-2.5833e+01],
         [-2.1812e+01],
         [-2.5912e+01],
         [-1.9507e+01],
         [-2.7119e+01],
         [-2.1757e+01],
         [-1.7117e+01],
         [-1.3100e+01],
         [-9.8422e+00],
         [-6.8812e+00],
         [-4.7680e-02],
         [ 4.0838e+00],
         [ 8.3293e+00],
         [ 1.0429e+01],
         [ 1.2877e+01],
         [ 1.0394e+01],
         [ 1.6613e+01],
         [ 1.5625e+01],
         [ 1.2923e+01],
         [ 1.1880e+01],
         [ 7.0214e+00],
         [-9.9242e-01],
         [-2.9318e+00],
         [-7.6519e+00],
         [-1.6216e+01],
         [-2.029